## 필요 라이브러리

In [ ]:
import os
import numpy as np
import pandas as pd
import mne
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from glob import glob

## 필요 함수 정의

### SuS Epoching

- SuS task EEG를 event 기준으로 epoching
- 현재는 이벤트 발생 시점의 -0.2s ~ 0.5s 를 epoching

In [ ]:
def epoch_sus_trials(eeg_file, events_file, tmin=-0.2, tmax=0.5):
    # Load EEG data and events
    raw = mne.io.read_raw_eeglab(eeg_file, preload=False)
    events_df = pd.read_csv(events_file, sep="\t")

    # Filter for stim_ON events
    # tsv 파일의 stim_ON 정보를 바탕으로 이벤트 시점 추출
    stim_events = events_df[events_df["value"] == "stim_ON"].reset_index(drop=True)

    # Create MNE events array
    # int array of shape (n_events, 3)
    # [onset_sample, 0, event_id] -> event_id는 1로 고정
    mne_events = []
    sfreq = raw.info["sfreq"]
    onsets = (stim_events["onset"].to_numpy(float) * sfreq).round().astype(int)
    mne_events = np.column_stack([
        onsets,
        np.zeros_like(onsets, dtype=int),
        np.full_like(onsets, 1, dtype=int),
    ])

    # Remove events too close to start or end
    pad_left = int(round(-tmin * sfreq))
    pad_right = int(round(tmax * sfreq))
    valid = (mne_events[:, 0] >= pad_left) & (mne_events[:, 0] < raw.n_times - pad_right)
    mne_events = mne_events[valid]
    mne_events = mne_events[np.argsort(mne_events[:, 0])]

    # Epoching
    # basline=(a,b) : 자극 전에 있던 평균 전위(a초 ~ b초 구간)를 빼서 0 근처로 맞춰주자
    epochs = mne.Epochs(
        raw,
        mne_events,
        event_id={"stim_ON": 1},
        tmin=tmin,
        tmax=tmax,
        baseline=(None, 0),
        detrend=1,
        preload=False,  # lazy loading
        reject_by_annotation=False,
        verbose=False,
    )

    # 필요한 시점에만 데이터를 메모리에 로드
    epochs.load_data()

    # numpy 형태로 반환
    X = epochs.get_data()  # (n_trials, n_channels, n_times)
    subj = os.path.basename(eeg_file).split("_")[0]
    return X, subj


### CCD RT

- 버튼을 누르지 않은 경우 제외 (+ 너무 빠르거나 너무 느리게 반응한 것 제외)
- 정답 피드백을 받은 trial만 남기기

In [ ]:
def parse_ccd_events(events_tsv):

    df = pd.read_csv(events_tsv, sep="\t")
    rts = []

    # target 이벤트만 선택
    targets = df[df["value"].isin(["left_target", "right_target"])].reset_index(drop=True)

    for _, target in targets.iterrows():
        t_start = target["onset"]

        # target 이후 첫 번째 buttonPress 찾기
        presses = df[(df["onset"] > t_start) & (df["value"].str.contains("buttonPress"))]
        if presses.empty:
            continue  # 버튼 안 누른 trial 제외
        first_press = presses.iloc[0]

        # buttonPress 이후 feedback 찾기
        feedbacks = df[(df["onset"] > first_press["onset"]) & (df["feedback"] == "smiley_face")]
        if feedbacks.empty:
            continue  # 정답 피드백 없는 trial 제외 (오답 또는 no-feedback)

        # reaction time 계산
        rt = first_press["onset"] - t_start

        # outlier 제거 (0.1s < RT < 3s)
        if 0.1 < rt < 3.0:
            rts.append(rt)

    return rts


### Matching

1. 동일 subject 내의 모든 SuS trial과 평균 CCD RT를 매칭

In [ ]:
def build_subject_level_data(sus_files, ccd_files):
    subj_erp = {}
    subj_rt = {}

    # SuS ERP
    for eeg_file, events_file in sus_files:
        X, subj = epoch_sus_trials(eeg_file, events_file)
        erp = X.mean(axis=0)  # ERP (C, T)
        if subj in subj_erp:
            subj_erp[subj].append(erp)
        else:
            subj_erp[subj] = [erp]

    subj_erp = {k: np.mean(v, axis=0) for k, v in subj_erp.items()}  # 평균 ERP

    # CCD mean RT
    for ev_file in ccd_files:
        subj = os.path.basename(ev_file).split("_")[0]
        rts = parse_ccd_events(ev_file)
        if len(rts) > 0:
            if subj in subj_rt:
                subj_rt[subj].extend(rts)
            else:
                subj_rt[subj] = rts

    subj_rt = {k: np.mean(v) for k, v in subj_rt.items()}  # 평균 RT

    # 공통 subject만 추출
    common_subj = set(subj_erp.keys()) & set(subj_rt.keys())
    X_all, y_all, subj_ids = [], [], []
    for subj in common_subj:
        X_all.append(subj_erp[subj])
        y_all.append(subj_rt[subj])
        subj_ids.append(subj)

    return np.array(X_all), np.array(y_all), np.array(subj_ids)


2. Task Invariant Feature Matching - channel 별 scale 과 band 구성을 맞춰주기

- CCD의 반응시간(또는 목표 값)을 회귀로 예측하면서, 동시에 과제(SuS vs CCD) 정보를 숨기는(invariant) 방향으로 Encoder를 학습(적대적 학습, GRL)
- 선택적으로 SuS/CCD 표현 분포를 유사하게 만드는 MMD 정렬까지 함께 적용.

*MMD : 두 task 간 latent feature 분포를 유사하게 만드는 것

In [ ]:
from scipy.signal import welch

def normalize_eeg(X):
    """채널별 Z-score 정규화 (task invariant scale)"""
    return (X - X.mean(axis=-1, keepdims=True)) / (X.std(axis=-1, keepdims=True) + 1e-6)

def compute_bandpower(X, sfreq=100, bands=((1,4),(4,8),(8,13),(13,30))):
    """
    Welch 파워스펙트럼을 통해 band-power 특징 추출
    X: (n_trials, n_channels, n_times)
    """
    bp = []
    for trial in X:
        f, Pxx = welch(trial, sfreq, nperseg=min(256, trial.shape[-1]))
        feats = []
        for (low, high) in bands:
            idx = np.logical_and(f >= low, f <= high)
            feats.append(Pxx[:, idx].mean(axis=1))
        bp.append(np.stack(feats, axis=1))  # (channels, n_bands)
    return np.array(bp)  # (n_trials, channels, n_bands)

def build_subject_level_data_invariant(sus_files, ccd_files, sfreq=100):
    subj_erp, subj_rt = {}, {}

    # --- SuS ERP ---
    for eeg_file, events_file in sus_files:
        X, subj = epoch_sus_trials(eeg_file, events_file)
        X = normalize_eeg(X)
        X_bp = compute_bandpower(X, sfreq=sfreq)
        erp = X_bp.mean(axis=0)  # 평균 band-power (C, n_bands)
        subj_erp.setdefault(subj, []).append(erp)
    subj_erp = {k: np.mean(v, axis=0) for k, v in subj_erp.items()}

    # --- CCD RT ---
    for ev_file in ccd_files:
        subj = os.path.basename(ev_file).split("_")[0]
        rts = parse_ccd_events(ev_file)
        if len(rts) > 0:
            subj_rt.setdefault(subj, []).extend(rts)
    subj_rt = {k: np.nanmean(v) for k, v in subj_rt.items()}

    # --- 매칭 ---
    common = set(subj_erp) & set(subj_rt)
    X_all, y_all, subj_ids = [], [], []
    for s in common:
        X_all.append(subj_erp[s].flatten())  # (C×n_bands)
        y_all.append(subj_rt[s])
        subj_ids.append(s)
    return np.array(X_all), np.array(y_all), np.array(subj_ids)


3. Representation Alignment - encoder 출력 공간 일치

MMD Loss : SuS와 CCD의 latent feature 분포 차를 penalty로 줌

### Meta 데이터 불러오기

In [ ]:
def load_all_participants(bids_root, datasets):
    """
    모든 release 폴더(ds005505 ~ ds005516)의 participants.tsv를 읽고
    subject ID 기준으로 통합 dataframe 생성
    """
    dfs = []
    for ds in datasets:
        pfile = os.path.join(bids_root, ds, "participants.tsv")
        if os.path.exists(pfile):
            df = pd.read_csv(pfile, sep="\t")
            dfs.append(df)
    df_all = pd.concat(dfs, ignore_index=True)
    df_all = df_all.drop_duplicates(subset=["participant_id"])
    df_all = df_all.set_index("participant_id")
    return df_all


In [ ]:
def extract_demographic_meta(df_all, subj_ids):
    meta_features = []
    for subj in subj_ids:
        if subj in df_all.index:
            row = df_all.loc[subj]
            # NaN-safe 변환 (없으면 0으로 대체)
            age = float(row["age"]) if pd.notna(row["age"]) else 0.0
            sex = 1 if row["sex"] == "M" else 0
            ehq = float(row["ehq_total"]) if pd.notna(row["ehq_total"]) else 0.0
            meta_features.append([age, sex, ehq])
        else:
            meta_features.append([0.0, 0.0, 0.0])  # 없는 경우 기본값
    return np.array(meta_features, dtype=np.float32)


## Dataset 정의

*Task Invariant Matching 기반으로 수정

In [ ]:
from torch.utils.data import Dataset

class SubjectDataset(Dataset):
    def __init__(self, X, meta=None, y=None, subj_ids=None, normalize=False):
        """
        Parameters
        ----------
        X : np.ndarray
            EEG feature (N, C, T) or (N, C, n_bands)
        meta : np.ndarray or None
            Demographic meta features (age, sex, ehq_total) (N, 3)
        y : np.ndarray or None
            Target values (N,)
        subj_ids : list or np.ndarray
            Subject IDs
        normalize : bool
            True이면 각 샘플별 z-score 정규화 적용
        """
        # --- 데이터 복사 및 정규화 ---
        if normalize:
            X = (X - X.mean(axis=-1, keepdims=True)) / (X.std(axis=-1, keepdims=True) + 1e-6)

        # --- 텐서 변환 ---
        self.X = torch.tensor(X, dtype=torch.float32)        # (N, C, T) or (N, C, n_bands)
        self.meta = torch.tensor(meta, dtype=torch.float32) if meta is not None else None
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1) if y is not None else None
        self.subj_ids = subj_ids if subj_ids is not None else np.arange(len(X))

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        sample = {"eeg": self.X[idx], "subject": self.subj_ids[idx]}
        if self.meta is not None:
            sample["meta"] = self.meta[idx]
        if self.y is not None:
            sample["label"] = self.y[idx]
        return sample


## 모델 정의

Sus, CCD task domain 차이를 줄여주는 메커니즘 필요
- Task-Invariant Encoder + Task Classifier (Adversarial Learning)
- Normalization & Statistical Matching 층 추가
- Feature-Level Regularization (MMD Loss 통합)
- Global Average Pooling 대체 → Spatial-Frequency Fusion
- Dropout 조정 / Stochastic Depth 사용

In [ ]:
# --------------------------------------------------------
# 1. Gradient Reversal Layer (for adversarial domain learning)
# --------------------------------------------------------
class GradientReversal(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, lambda_):
        ctx.lambda_ = lambda_
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return -ctx.lambda_ * grad_output, None


class GRL(nn.Module):
    def __init__(self, lambda_=1.0):
        super().__init__()
        self.lambda_ = lambda_

    def forward(self, x):
        return GradientReversal.apply(x, self.lambda_)


# --------------------------------------------------------
# 2. EEGNeX_Invariant model
# --------------------------------------------------------
class EEGNeX_Invariant(nn.Module):
    """
    EEG Foundation Challenge 2025
    Cross-task generalization version of EEGNeX.

    - Task-invariant FC encoder (band-power or normalized feature)
    - InstanceNorm for scale invariance
    - Meta branch fusion
    - Optional domain discriminator for adversarial alignment
    """

    def __init__(
        self,
        in_dim,            # flattened EEG feature dimension (e.g., C×n_bands)
        d_meta=3,
        feat_dim=128,
        dropout_p=0.3,
        use_domain_adv=True,
        grl_lambda=1.0,
    ):
        super().__init__()
        self.use_domain_adv = use_domain_adv

        # -------- EEG encoder (task-invariant) --------
        self.encoder = nn.Sequential(
            nn.Linear(in_dim, 256),
            nn.ReLU(),
            nn.InstanceNorm1d(256, affine=True),   # InstanceNorm 적용
            nn.Dropout(dropout_p),
            nn.Linear(256, feat_dim),
            nn.ReLU(),
        )

        # -------- Meta branch --------
        self.fc_meta = nn.Sequential(
            nn.Linear(d_meta, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU()
        )

        # -------- Regression head (RT prediction) --------
        self.reg_head = nn.Sequential(
            nn.Dropout(dropout_p),
            nn.Linear(feat_dim + 8, 1)
        )

        # -------- Domain adversarial head (optional) --------
        if self.use_domain_adv:
            self.grl = GRL(lambda_=grl_lambda)
            self.domain_disc = nn.Sequential(
                nn.Linear(feat_dim, 64),
                nn.ReLU(),
                nn.Linear(64, 2)   # 2 tasks: SuS / CCD
            )

    # --------------------------------------------------
    # forward pass
    # --------------------------------------------------
    def forward(self, x, meta, return_feature=False):
        """
        Parameters
        ----------
        x : EEG feature (B, in_dim)
        meta : demographic feature (B, d_meta)
        return_feature : if True, returns latent EEG feature for MMD loss
        """
        eeg_feat = self.encoder(x)             # (B, feat_dim)
        meta_feat = self.fc_meta(meta)         # (B, 8)
        fused = torch.cat([eeg_feat, meta_feat], dim=1)
        rt_pred = self.reg_head(fused)         # (B, 1)

        if not self.use_domain_adv:
            return rt_pred if not return_feature else (rt_pred, eeg_feat)

        # Domain adversarial branch
        rev_feat = self.grl(eeg_feat)
        domain_logits = self.domain_disc(rev_feat)   # (B, 2)

        if return_feature:
            return rt_pred, domain_logits, eeg_feat
        else:
            return rt_pred, domain_logits


# --------------------------------------------------------
# 3. MMD Loss Function (for representation alignment)
# --------------------------------------------------------
def mmd_loss(source, target, sigma=1.0):
    """Maximum Mean Discrepancy for SuS↔CCD alignment"""
    def gaussian_kernel(x, y, sigma):
        x_norm = (x ** 2).sum(1).view(-1, 1)
        y_norm = (y ** 2).sum(1).view(1, -1)
        dist = x_norm + y_norm - 2 * torch.mm(x, y.t())
        return torch.exp(-dist / (2 * sigma ** 2))

    Kxx = gaussian_kernel(source, source, sigma)
    Kyy = gaussian_kernel(target, target, sigma)
    Kxy = gaussian_kernel(source, target, sigma)
    return Kxx.mean() + Kyy.mean() - 2 * Kxy.mean()


## 학습 루프

- Regression (MSE) + Domain Adversarial (CE) + MMD
- Gradient clipping

In [ ]:
def train_model_cross_task(
    model,
    sus_loader,             # Source domain: SuS (EEG only)
    ccd_loader,             # Target domain: CCD (EEG + RT)
    n_epochs=20,
    lr=1e-3,
    device="cuda",
    lambda_domain=0.1,
    lambda_mmd=0.1,
):
    """
    Cross-task training loop for EEGNeX_Invariant model.
    - SuS: source domain (no RT label, domain alignment only)
    - CCD: target domain (RT label supervised learning)
    """
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    mse_loss = nn.MSELoss()

    for epoch in range(n_epochs):
        model.train()
        total_loss, reg_loss, dom_loss, mmd_loss_val = 0.0, 0.0, 0.0, 0.0

        progress = tqdm(zip(sus_loader, ccd_loader), total=min(len(sus_loader), len(ccd_loader)),
                        desc=f"Epoch {epoch+1}/{n_epochs}")

        for sus_batch, ccd_batch in progress:
            # -------------------------------
            # Load data
            # -------------------------------
            sus_x = sus_batch["eeg"].to(device)
            sus_meta = sus_batch["meta"].to(device)

            ccd_x = ccd_batch["eeg"].to(device)
            ccd_meta = ccd_batch["meta"].to(device)
            ccd_y = ccd_batch["label"].to(device)

            # -------------------------------
            # Forward
            # -------------------------------
            rt_pred_sus, domain_logits_sus, feat_sus = model(sus_x, sus_meta, return_feature=True)
            rt_pred_ccd, domain_logits_ccd, feat_ccd = model(ccd_x, ccd_meta, return_feature=True)

            # -------------------------------
            # Loss components
            # -------------------------------
            # (1) Regression loss on CCD
            loss_reg = mse_loss(rt_pred_ccd, ccd_y)

            # (2) Domain adversarial loss
            domain_labels = torch.cat([
                torch.zeros(len(domain_logits_sus)),  # 0 = SuS
                torch.ones(len(domain_logits_ccd))    # 1 = CCD
            ], dim=0).long().to(device)
            domain_logits = torch.cat([domain_logits_sus, domain_logits_ccd], dim=0)
            loss_domain = F.cross_entropy(domain_logits, domain_labels)

            # (3) MMD loss (feature alignment)
            def mmd_loss(source, target, sigma=1.0):
                def gaussian_kernel(x, y, sigma):
                    x_norm = (x ** 2).sum(1).view(-1, 1)
                    y_norm = (y ** 2).sum(1).view(1, -1)
                    dist = x_norm + y_norm - 2 * torch.mm(x, y.t())
                    return torch.exp(-dist / (2 * sigma ** 2))
                Kxx = gaussian_kernel(source, source, sigma)
                Kyy = gaussian_kernel(target, target, sigma)
                Kxy = gaussian_kernel(source, target, sigma)
                return Kxx.mean() + Kyy.mean() - 2 * Kxy.mean()

            loss_mmd = mmd_loss(feat_sus, feat_ccd)

            # (4) Total loss
            loss = loss_reg + lambda_domain * loss_domain + lambda_mmd * loss_mmd

            # -------------------------------
            # Backprop
            # -------------------------------
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
            optimizer.step()

            # -------------------------------
            # Accumulate metrics
            # -------------------------------
            total_loss += loss.item()
            reg_loss += loss_reg.item()
            dom_loss += loss_domain.item()
            mmd_loss_val += loss_mmd.item()

        # -------------------------------
        # Log
        # -------------------------------
        n_batches = min(len(sus_loader), len(ccd_loader))
        print(
            f"[Epoch {epoch+1}/{n_epochs}] "
            f"Total={total_loss/n_batches:.4f} | "
            f"Reg={reg_loss/n_batches:.4f} | "
            f"Dom={dom_loss/n_batches:.4f} | "
            f"MMD={mmd_loss_val/n_batches:.4f}"
        )

        # -------------------------------
        # Validation (optional)
        # -------------------------------
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in ccd_loader:
                x, m, y = batch["eeg"].to(device), batch["meta"].to(device), batch["label"].to(device)
                pred, _ = model(x, m)
                val_loss += mse_loss(pred, y).item() * len(x)
        val_loss /= len(ccd_loader.dataset)
        print(f"Validation Loss: {val_loss:.4f}")

    return model


## 실제 학습 코드

1. raw 데이터 버전

In [ ]:
# ds005505 ~ ds005516
all_datasets = [f"ds00{i}" for i in range(5505, 5517)]

# Train: Release5(ds005509) 제외
train_datasets = [ds for ds in all_datasets if ds != "ds005509"]
val_datasets   = ["ds005509"]

BIDS_ROOT = "/data5/open_data/HBN/EEG_BIDS"

def collect_sus_files(datasets):
    all_files = []
    for ds in datasets:
        bids_root = os.path.join(BIDS_ROOT, ds)
        eeg_files = glob(os.path.join(bids_root, "sub-*", "eeg", f"sub-*_task-surroundSupp_run-*_eeg.set"))
        for eeg_file in eeg_files:
            events_file = eeg_file.replace("_eeg.set", "_events.tsv")
            all_files.append((eeg_file, events_file))
    return all_files

def collect_ccd_files(datasets):
    all_files = []
    for ds in datasets:
        bids_root = os.path.join(BIDS_ROOT, ds)
        ev_files = glob(os.path.join(bids_root, "sub-*", "eeg", f"sub-*_task-contrastChangeDetection_run-*_events.tsv"))
        all_files.extend(ev_files)
    return all_files



# 1. participants.tsv 병합
df_participants = load_all_participants(BIDS_ROOT, train_datasets)

# 2. SuS & CCD 파일 수집
sus_train_files = collect_sus_files(train_datasets)
ccd_train_files = collect_ccd_files(train_datasets)
sus_val_files   = collect_sus_files(val_datasets)
ccd_val_files   = collect_ccd_files(val_datasets)

2. preprocessing data 버전
- bad channel을 그대로 두면 그것을 잘못 학습할 가능성이 있음
- 대부분의 cognitive task ERP 성분 (P1, N1, P3 등)은 0.5–30 Hz 범위 -> 100 Hz sampling이면 Nyquist 기준 충분히 커버 가능 (고주파는 오히려 근전도 등의 노이즈 O)

In [ ]:
def collect_preprocessed_files_by_task(root_path):
    """
    전처리된 EEG 파일 중 SurroundSupp(=SuS), ContrastChangeDetection(=CCD) task만 수집.

    Parameters
    ----------
    root_path : str
        전처리된 EEG 데이터가 저장된 최상위 폴더 경로
        예: "/data5/open_data/HBN/Preprocessed_EEG/0922try_bySubject/"

    Returns
    -------
    sus_files : list of (eeg_file, "")
        surroundSupp task 파일 목록
    ccd_files : list of (eeg_file, "")
        contrastChangeDetection task 파일 목록
    """
    print(f"\n📂 Collecting preprocessed EEG files from: {root_path}")

    # Surround Suppression (SuS)
    sus_pattern = os.path.join(root_path, "sub-*", "*_task-surroundSupp_*_eeg_pp.set")
    sus_files = glob(sus_pattern, recursive=True)
    print(f"  - Found {len(sus_files)} surroundSupp files")

    # Contrast Change Detection (CCD)
    ccd_pattern = os.path.join(root_path, "sub-*", "*_task-contrastChangeDetection_*_eeg_pp.set")
    ccd_files = glob(ccd_pattern, recursive=True)
    print(f"  - Found {len(ccd_files)} contrastChangeDetection files")

    # (파일 경로, events_file 자리 placeholder) 형태로 반환
    sus_pairs = [(f, "") for f in sus_files]
    ccd_pairs = [(f, "") for f in ccd_files]
    return sus_pairs, ccd_pairs


In [ ]:
PREPROCESSED_ROOT = "/data5/open_data/HBN/Preprocessed_EEG/0922try_bySubject/" 
BIDS_ROOT = "/data5/open_data/HBN/EEG_BIDS"

# --- 데이터셋 그룹 정의 ---
train_datasets = [f"ds00{i}" for i in range(5505, 5517) if i != 5509]
val_datasets   = ["ds005509"]

# --- participants.tsv 병합 (label 정보)
df_train_participants = load_all_participants(BIDS_ROOT, train_datasets)
df_val_participants   = load_all_participants(BIDS_ROOT, val_datasets)

# --- 전처리된 EEG 파일 수집 ---
sus_all_files, ccd_all_files = collect_preprocessed_files_by_task(PREPROCESSED_ROOT)

# participant ID 기준으로 train/val 분리
train_subjects = set(df_train_participants["participant_id"])
val_subjects   = set(df_val_participants["participant_id"])

def split_by_subject(file_list, train_subj, val_subj):
    train_files, val_files = [], []
    for f, ev in file_list:
        subj = os.path.basename(f).split("_")[0]  # e.g., "sub-005504"
        subj_id = subj.replace("sub-", "")
        if subj_id in train_subj:
            train_files.append((f, ev))
        elif subj_id in val_subj:
            val_files.append((f, ev))
    return train_files, val_files

# SuS
sus_train_files, sus_val_files = split_by_subject(sus_all_files, train_subjects, val_subjects)
# CCD
ccd_train_files, ccd_val_files = split_by_subject(ccd_all_files, train_subjects, val_subjects)

print(f"SuS: Train={len(sus_train_files)}, Val={len(sus_val_files)}")
print(f"CCD: Train={len(ccd_train_files)}, Val={len(ccd_val_files)}")


In [ ]:
# --------------------------------------------------------------
# SuS (source domain) 데이터 생성
# --------------------------------------------------------------
print("\n🔹 Building SuS EEG feature dataset (source domain)...")
X_sus, _, subj_sus = build_subject_level_data_invariant(
    sus_train_files, ccd_train_files=[], sfreq=100
)  # CCD 파일은 필요 없음
meta_sus = extract_demographic_meta(df_train_participants.set_index("participant_id"), subj_sus)

# --------------------------------------------------------------
# CCD (target domain) 데이터 생성
# --------------------------------------------------------------
print("\n🔹 Building CCD EEG feature + RT dataset (target domain)...")
X_ccd, y_ccd, subj_ccd = build_subject_level_data_invariant(
    sus_files=[], ccd_files=ccd_train_files, sfreq=100
)
meta_ccd = extract_demographic_meta(df_train_participants.set_index("participant_id"), subj_ccd)

# --------------------------------------------------------------
# 메타 스케일링 (train에서 fit)
# --------------------------------------------------------------
def fit_meta_scaler(meta_train):
    mean = meta_train.mean(axis=0)
    std = meta_train.std(axis=0) + 1e-6
    mean[1], std[1] = 0.0, 1.0  # sex 제외
    return {'mean': mean, 'std': std}

def apply_meta_scaler(meta, scaler):
    return (meta - scaler['mean']) / scaler['std']

scaler_meta = fit_meta_scaler(meta_ccd)
meta_sus_z = apply_meta_scaler(meta_sus, scaler_meta)
meta_ccd_z = apply_meta_scaler(meta_ccd, scaler_meta)

# --------------------------------------------------------------
# Dataset/Dataloader 정의
# --------------------------------------------------------------
sus_dataset = SubjectDataset(X_sus, meta_sus_z, y=None, subj_ids=subj_sus, normalize=True)
ccd_dataset = SubjectDataset(X_ccd, meta_ccd_z, y_ccd, subj_ccd, normalize=True)

sus_loader = DataLoader(sus_dataset, batch_size=32, shuffle=True, num_workers=4)
ccd_loader = DataLoader(ccd_dataset, batch_size=32, shuffle=True, num_workers=4)

print(f"SuS loader: {len(sus_loader)} batches, CCD loader: {len(ccd_loader)} batches")


### 데이터 저장

In [ ]:
SAVE_DIR = "./prepared_datasets_cross_task"
os.makedirs(SAVE_DIR, exist_ok=True)

# SuS 데이터 저장 (EEG + meta + subj_ids)
np.savez_compressed(
    os.path.join(SAVE_DIR, "sus_source_train.npz"),
    X=X_sus,
    meta=meta_sus_z,
    subj_ids=subj_sus
)
print(f"✅ Saved SuS source data: {X_sus.shape}")

# CCD 데이터 저장 (EEG + meta + y + subj_ids)
np.savez_compressed(
    os.path.join(SAVE_DIR, "ccd_target_train.npz"),
    X=X_ccd,
    y=y_ccd,
    meta=meta_ccd_z,
    subj_ids=subj_ccd
)
print(f"✅ Saved CCD target data: {X_ccd.shape}, labels: {y_ccd.shape}")

### 데이터 로드

In [ ]:
# ----------------------------------------------------------
# 데이터 로드
# ----------------------------------------------------------
sus_data = np.load("./prepared_datasets_cross_task/sus_source_train.npz", allow_pickle=True)
ccd_data = np.load("./prepared_datasets_cross_task/ccd_target_train.npz", allow_pickle=True)

X_sus = sus_data["X"]
meta_sus_z = sus_data["meta"]
subj_sus = sus_data["subj_ids"]

X_ccd = ccd_data["X"]
y_ccd = ccd_data["y"]
meta_ccd_z = ccd_data["meta"]
subj_ccd = ccd_data["subj_ids"]

print(" Loaded SuS:", X_sus.shape, "  CCD:", X_ccd.shape)

# ----------------------------------------------------------
# 2️⃣ Torch Dataset / DataLoader 구성
# ----------------------------------------------------------
sus_dataset = SubjectDataset(X_sus, meta_sus_z, y=None, subj_ids=subj_sus, normalize=False)
ccd_dataset = SubjectDataset(X_ccd, meta_ccd_z, y_ccd, subj_ccd, normalize=False)

sus_loader = DataLoader(sus_dataset, batch_size=32, shuffle=True, num_workers=4)
ccd_loader = DataLoader(ccd_dataset, batch_size=32, shuffle=True, num_workers=4)

print(f"SuS loader: {len(sus_loader)} batches, CCD loader: {len(ccd_loader)} batches")


### 모델 학습

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
in_dim = X_ccd.shape[1] * X_ccd.shape[2]  # (C × n_bands)

model = EEGNeX_Invariant(
    in_dim=in_dim,
    d_meta=3,
    feat_dim=128,
    dropout_p=0.3,
    use_domain_adv=True,
    grl_lambda=1.0
).to(DEVICE)

model = train_model_cross_task(
    model=model,
    sus_loader=sus_loader,   # SuS: source (domain alignment only)
    ccd_loader=ccd_loader,   # CCD: target (RT supervised)
    n_epochs=30,
    lr=1e-3,
    device=DEVICE,
    lambda_domain=0.1,
    lambda_mmd=0.1,
)

# 모델 저장
torch.save({"model_state_dict": model.state_dict()}, "./checkpoints/challenge1_eegnex_invariant_best.pt")
print("Saved trained model to ./checkpoints/eegnex_invariant_best.pt")


Epoch 1/100: 100%|██████████| 105/105 [00:04<00:00, 21.38it/s]


Epoch 1: Train Loss=9.7219, Val Loss=4.8655


Epoch 2/100: 100%|██████████| 105/105 [00:01<00:00, 83.81it/s] 


Epoch 2: Train Loss=8.5783, Val Loss=4.9244


Epoch 3/100: 100%|██████████| 105/105 [00:01<00:00, 85.89it/s]


...

Epoch 98: Train Loss=4.3377, Val Loss=6.6366


Epoch 99/100: 100%|██████████| 105/105 [00:01<00:00, 100.90it/s]


Epoch 99: Train Loss=4.2630, Val Loss=13.3879


Epoch 100/100: 100%|██████████| 105/105 [00:01<00:00, 100.26it/s]


Epoch 100: Train Loss=4.3881, Val Loss=7.5331
